In [6]:
import os
import cv2
from PIL import Image
from numba import jit
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pingouin as pg
import scipy
from collections import OrderedDict
import sklearn.cluster
from collections import Counter
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score,f1_score, roc_curve, auc

In [7]:
draw_classes = ['ASD', 'TD']
draw_num = [24, 10]

In [8]:
horizon_intervals = [[],[]]
vertical_intervals = [[],[]]

grid_interval = 20
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

for i,cls in enumerate(draw_classes):
    print(cls)
    draw_dr = f'../paintings/valid/{cls}'
    for j in range(1,draw_num[i]+1):
        print(j)
        im = cv2.imread(f"{draw_dr}/{j}.jpg")
        im1 = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im2 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im3 = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
        im_min = np.minimum(im2[:,:,0],np.minimum(im2[:,:,1],im2[:,:,2]))
        
        ret,im_bin = cv2.threshold(im_min,160,255,cv2.THRESH_BINARY)
        im_closed = cv2.morphologyEx(im_bin, cv2.MORPH_CLOSE, kernel,iterations=2)
        draw_pix = np.array(np.where(im_closed == 0)).T
        bd_rect = cv2.boundingRect(draw_pix)
        
        x1 = bd_rect[1]; x2 = x1+bd_rect[3]
        y1 = bd_rect[0]; y2 = y1+bd_rect[2]
        s1 = bd_rect[3]; s2 = bd_rect[2]
        
        # vertical test lines
        cnt = 0
        for x in range(x1, x2, grid_interval):
            series = im1[:,x]
            lowpass = scipy.signal.filtfilt(*scipy.signal.butter(8, 0.1, 'lowpass'), series)

            peaks, _ = scipy.signal.find_peaks(lowpass)
            basins, _ = scipy.signal.find_peaks(-lowpass)
            for y in peaks:
                if not(y1 <= y <= y2): continue
                fd = basins[basins < y]
                if fd.shape[0] == 0: continue
                nearest_basin = fd[-1]
                if im3[nearest_basin,x,1] > 150 and (lowpass[y] - lowpass[nearest_basin]) > 35 and (y-nearest_basin) > 5 and im3[y, x, 2] > 140:
                    cnt += 1
        cnt = cnt / (s2 * len(list(range(x1, x2, grid_interval))))
        vertical_intervals[i].append(cnt)
        
        # horizontal test lines
        cnt = 0
        for y in range(y1, y2, grid_interval):
            series = im1[y,:]
            lowpass = scipy.signal.filtfilt(*scipy.signal.butter(8, 0.1, 'lowpass'), series)

            peaks, _ = scipy.signal.find_peaks(lowpass)
            basins, _ = scipy.signal.find_peaks(-lowpass)
            for x in peaks:
                if not(x1 <= x <= x2): continue
                fd = basins[basins < x]
                if fd.shape[0] == 0: continue
                nearest_basin = fd[-1]
                if im3[y,nearest_basin,1] > 150 and (lowpass[x] - lowpass[nearest_basin]) > 35 and (y-nearest_basin) > 5 and im3[y, x, 2] > 140:
                    cnt += 1
        cnt = cnt / (s1 * len(list(range(y1, y2, grid_interval))))
        horizon_intervals[i].append(cnt)

ASD
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
TD
1
2
3
4
5
6
7
8
9
10


In [9]:
horizon_intervals = np.array(horizon_intervals, dtype=object)
vertical_intervals = np.array(vertical_intervals, dtype=object)
np.save('./results/valid_horizon_intervals.npy', horizon_intervals)
np.save('./results/valid_vertival_intervals.npy', vertical_intervals)

In [11]:
areas = [[],[]]


grid_interval = 20
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

for i,cls in enumerate(draw_classes):
    print(cls)
    draw_dr = f'../paintings/valid/{cls}/'
    for j in range(1,draw_num[i]+1):
        print(j)
        im = cv2.imread(f"{draw_dr}/{j}.jpg")
        im1 = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im2 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im3 = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
        im_min = np.minimum(im2[:,:,0],np.minimum(im2[:,:,1],im2[:,:,2]))
        
        ret,im_bin = cv2.threshold(im_min,160,255,cv2.THRESH_BINARY)
        im_closed = cv2.morphologyEx(im_bin, cv2.MORPH_CLOSE, kernel,iterations=2)
        draw_pix = np.array(np.where(im_closed == 0)).T
        bd_rect = cv2.boundingRect(draw_pix)
        
        x1 = bd_rect[1]; x2 = x1+bd_rect[3]
        y1 = bd_rect[0]; y2 = y1+bd_rect[2]
        s1 = bd_rect[3]; s2 = bd_rect[2]
        
        area = s1*s2 / (im.shape[0] * im.shape[1])

        areas[i].append(area)

ASD
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
TD
1
2
3
4
5
6
7
8
9
10


In [12]:
areas = np.array(areas, dtype=object)
np.save('./results/valid_areas.npy', areas)

In [13]:
def HSV2cone(points):
    points = points.copy().astype(np.double)
    points[:,0] = 2 * np.deg2rad(points[:,0])
    points[:,1] /=255; points[:,2] /= 255    # uniformization
    
    new_points = np.zeros(points.shape, dtype=np.double)
    points_sv = points[:, 1] * points[:, 2]
    
    new_points[:,0] = points_sv * np.cos(points[:, 0])
    new_points[:,1] = points_sv * np.sin(points[:, 0])
    new_points[:,2] = points[:, 2]
    return new_points
    
def cone2HSV(points):
    new_points = np.zeros(points.shape, dtype=np.double)
    new_points[:,0] = np.rad2deg(np.mod(np.arctan2(points[:,1], points[:,0]),2*np.pi))/2
    new_points[:,1] = np.sqrt((np.square(points[:,0])+np.square(points[:,1]))/np.square(points[:,2])) * 255
    new_points[:,2] = points[:,2] * 255
    
    return new_points.astype(np.uint8)

In [16]:
tone_HSVs = [[],[]]

n_clusters = 6
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

for i,cls in enumerate(draw_classes):
    print(cls)
    draw_dr = f'../paintings/valid/{cls}/'
    for j in range(1,draw_num[i]+1):
        print(j)
        im = cv2.imread(f"{draw_dr}/{j}.jpg")
        im2 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im3 = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

        im_min = np.minimum(im2[:,:,0],np.minimum(im2[:,:,1],im2[:,:,2]))

        ret,im_bin = cv2.threshold(im_min,160,255,cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
        im_closed = cv2.morphologyEx(im_bin, cv2.MORPH_CLOSE, kernel,iterations=2)

        draw_pix = np.array(np.where(im_closed == 0)).T
        bd_rect = cv2.boundingRect(draw_pix)

        x1 = bd_rect[1]; x2 = x1+bd_rect[3]
        y1 = bd_rect[0]; y2 = y1+bd_rect[2]

        pixel_set_hsv = im3[y1:y2+1, x1:x2+1, :].copy()
        pixel_set_hsv = pixel_set_hsv.reshape((pixel_set_hsv.shape[0]*pixel_set_hsv.shape[1],3))

        # Cone transformation
        pixel_set_cone = HSV2cone(pixel_set_hsv)

        kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters, n_init = 'auto')
        kmeans.fit(pixel_set_cone)

        histo = np.histogram(kmeans.labels_, bins=n_clusters, range=(0, n_clusters))
        percent = histo[0] / pixel_set_cone.shape[0] * 100
        tone_idx = percent.argmax()

        centers_HSV = cone2HSV(kmeans.cluster_centers_)
        tone_HSV = centers_HSV[tone_idx]
        
        tone_HSVs[i].append(tone_HSV)

ASD
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
TD
1
2
3
4
5
6
7
8
9
10


In [17]:
tone_HSVs = np.array(tone_HSVs, dtype=object)
np.save('./results/valid_tone_HSVs.npy', tone_HSVs)

In [20]:
def min_distance(points):
    return np.min(scipy.spatial.distance.pdist(points))

In [21]:
color_clusters = [[],[]]

kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

for i,cls in enumerate(draw_classes):
    print(cls)
    draw_dr = f'../paintings/valid/{cls}/'
    for j in range(1,draw_num[i]+1):
        print(j)
        im = cv2.imread(f"{draw_dr}/{j}.jpg")
        im2 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im3 = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

        im_min = np.minimum(im2[:,:,0],np.minimum(im2[:,:,1],im2[:,:,2]))

        ret,im_bin = cv2.threshold(im_min,160,255,cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
        im_closed = cv2.morphologyEx(im_bin, cv2.MORPH_CLOSE, kernel,iterations=2)

        draw_pix = np.array(np.where(im_closed == 0)).T
        bd_rect = cv2.boundingRect(draw_pix)

        x1 = bd_rect[1]; x2 = x1+bd_rect[3]
        y1 = bd_rect[0]; y2 = y1+bd_rect[2]

        pixel_set_hsv = im3[y1:y2+1, x1:x2+1, :].copy()
        pixel_set_hsv = pixel_set_hsv.reshape((pixel_set_hsv.shape[0]*pixel_set_hsv.shape[1],3))

        # Cone transformation
        pixel_set_cone = HSV2cone(pixel_set_hsv)

        n_clusters = 2
        while True:
            kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters, n_init = 'auto')
            kmeans.fit(pixel_set_cone)
            min_dis = min_distance(kmeans.cluster_centers_)
            if min_dis < 0.28:
                n_clusters -= 1
                break
            n_clusters += 1
        color_clusters[i].append(n_clusters)

ASD
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
TD
1
2
3
4
5
6
7
8
9
10


In [22]:
color_clusters = np.array(color_clusters, dtype=object)
np.save('./results/valid_color_clusters.npy', color_clusters)

In [23]:
### Thanks to github.com/zhaoxuhui

def calcEntropy(img):
    entropy = []

    hist = cv2.calcHist([img], [0], None, [256], [0, 255])
    total_pixel = img.shape[0] * img.shape[1]

    for item in hist:
        probability = item / total_pixel
        if probability == 0:
            en = 0
        else:
            en = -1 * probability * np.log(probability)[0]
        entropy.append(en)

    sum_en = np.sum(entropy)[0]
    return sum_en

@jit(nopython=True)
def calcIJ(img_patch):
    total_p = img_patch.shape[0] * img_patch.shape[1]
    if total_p % 2 != 0:
        center_p = img_patch[int(img_patch.shape[0] / 2), int(img_patch.shape[1] / 2)]
        mean_p = (np.sum(img_patch) - center_p) / (total_p - 1)
        return (center_p, mean_p)
    else:
        pass


def calcEntropy2dSpeedUp(img, win_w=3, win_h=3):
    height = img.shape[0]

    ext_x = int(win_w / 2)
    ext_y = int(win_h / 2)

    ext_h_part = np.zeros([height, ext_x], img.dtype)
    tem_img = np.hstack((ext_h_part, img, ext_h_part))
    ext_v_part = np.zeros([ext_y, tem_img.shape[1]], img.dtype)
    final_img = np.vstack((ext_v_part, tem_img, ext_v_part))

    new_width = final_img.shape[1]
    new_height = final_img.shape[0]

    # the most time-consuming step, traversing and computing the pairs
    IJ = []
    for i in range(ext_x, new_width - ext_x):
        for j in range(ext_y, new_height - ext_y):
            patch = final_img[j - ext_y:j + ext_y + 1, i - ext_x:i + ext_x + 1]
            ij = calcIJ(patch)
            IJ.append(ij)

    Fij = Counter(IJ).items()

    # the second most time-consuming step, calculating the probabilities of each pair occurring.
    Pij = []
    for item in Fij:
        Pij.append(item[1] * 1.0 / (new_height * new_width))

    H_tem = []
    for item in Pij:
        h_tem = -item * np.log(item)
        H_tem.append(h_tem)

    H = np.sum(H_tem)
    return H

In [24]:
entropy_grey = [[],[]]

for i,cls in enumerate(draw_classes):
    print(cls)
    draw_dr = f'../paintings/valid/{cls}/'
    for j in range(1,draw_num[i]+1):
        print(j)
        im_grey = cv2.imread(f"{draw_dr}/{j}.jpg",cv2.IMREAD_GRAYSCALE)
        ep = calcEntropy(im_grey)
        entropy_grey[i].append(ep)

ASD
1
2
3
4
5
6
7
8
9
10


D:\ProgramData\Anaconda3\envs\torch\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


11
12
13
14
15
16
17
18
19
20
21
22
23
24
TD
1
2
3
4
5
6
7
8
9
10


In [27]:
entropy2d_grey = [[],[]]

for i,cls in enumerate(draw_classes):
    print(cls)
    draw_dr = f'../paintings/valid/{cls}/'
    for j in range(1,draw_num[i]+1):
        print(j)
        im_grey = cv2.imread(f"{draw_dr}/{j}.jpg",cv2.IMREAD_GRAYSCALE)
        ep = calcEntropy2dSpeedUp(im_grey)
        entropy2d_grey[i].append(ep)

ASD
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
TD
1
2
3
4
5
6
7
8
9
10


In [28]:
entropy_grey = np.array(entropy_grey, dtype=object)
entropy2d_grey = np.array(entropy2d_grey, dtype=object)
np.save('./results/valid_entropy_grey.npy', entropy_grey)
np.save('./results/valid_entropy2d_grey.npy', entropy2d_grey)

In [29]:
def project_hue(im_HSV, im_gray):
    im_proj = np.zeros((im_HSV.shape[0],im_HSV.shape[1]),dtype=np.int16)
#     for i in range(im_HSV.shape[0]):
#         for j in range(im_HSV.shape[1]):
#             if im_HSV[i,j,2] < 20:
#                 im_proj[i,j] = -256
#             else:
#                 if im_HSV[i,j,1] < 15:
#                     im_proj[i,j] = -(255 - im_gray[i,j])
#                 else:
#                     im_proj[i,j] = im_HSV[i,j,0]
                    
    con1 = im_HSV[:,:,2] < 15
    con1r = ~con1
    con2 = im_HSV[:,:,1] < 10
    con2r = ~con2
    im_proj[con1] = -256
    im_proj[con1r & con2] = -(255 - im_gray[con1r & con2])
    im_proj[con1r & con2r] = im_HSV[:,:,0][con1r & con2r]
    return im_proj

In [42]:
def calcEntropy_hue(img):
    entropy = []

    hist = Counter(img.reshape(-1)).items()
    total_pixel = img.shape[0] * img.shape[1]

    for item in hist:
        probability = item[1] / total_pixel
        en = -1 * probability * np.log(probability)
        entropy.append(en)

    sum_en = np.sum(entropy)
    return sum_en

In [43]:
entropy_hue = [[],[]]

for i,cls in enumerate(draw_classes):
    print(cls)
    draw_dr = f'../paintings/experiment/{cls}/'
    for j in range(1,draw_num[i]+1):
        print(j)
        im = cv2.imread(f"{draw_dr}/{j}.jpg")
        im1 = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im3 = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
        
        im_proj = project_hue(im3, im1)
        ep = calcEntropy_hue(im_proj)
        entropy_hue[i].append(ep)

ASD
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
TD
1
2
3
4
5
6
7
8
9
10


In [45]:
entropy_hue = np.array(entropy_hue, dtype=object)
np.save('./results/valid_entropy_hue.npy', entropy_hue)

In [12]:
def HSV2cone(points):
    points = points.copy().astype(np.double)
    points[:,0] = 2 * np.deg2rad(points[:,0])
    points[:,1] /=255; points[:,2] /= 255    # uniformization
    
    new_points = np.zeros(points.shape, dtype=np.double)
    points_sv = points[:, 1] * points[:, 2]
    
    new_points[:,0] = points_sv * np.cos(points[:, 0])
    new_points[:,1] = points_sv * np.sin(points[:, 0])
    new_points[:,2] = points[:, 2]
    return new_points
    
def cone2HSV(points):
    new_points = np.zeros(points.shape, dtype=np.double)
    new_points[:,0] = np.rad2deg(np.mod(np.arctan2(points[:,1], points[:,0]),2*np.pi))/2
    new_points[:,1] = np.sqrt((np.square(points[:,0])+np.square(points[:,1]))/np.square(points[:,2])) * 255
    new_points[:,2] = points[:,2] * 255
    
    return new_points.astype(np.uint8)

In [13]:
entropy_grey = np.load('./results/valid_entropy_grey.npy', allow_pickle=True)
entropy2d_grey = np.load('./results/valid_entropy2d_grey.npy', allow_pickle=True)
entropy_hue = np.load('./results/valid_entropy_hue.npy', allow_pickle=True)
tone_HSVs = np.load('./results/valid_tone_HSVs.npy', allow_pickle=True)    # special handle
color_clusters = np.load('./results/valid_color_clusters.npy', allow_pickle=True)
areas = np.load('./results/valid_areas.npy', allow_pickle=True)
horizon_intervals = np.load('./results/valid_horizon_intervals.npy', allow_pickle=True)
vertical_intervals = np.load('./results/valid_vertival_intervals.npy', allow_pickle=True)

for feature in [entropy_grey, entropy2d_grey, entropy_hue, tone_HSVs, color_clusters, areas, horizon_intervals, vertical_intervals]:
    feature[0] = [v for i, v in enumerate(feature[0]) if i+1 not in {177,205,257,302}]
    feature[1] = [v for i, v in enumerate(feature[1]) if i+1 not in {78,79}]

features = [entropy_grey, entropy2d_grey, entropy_hue, color_clusters, areas, horizon_intervals, vertical_intervals]

In [14]:
ASD_HSV = np.array(tone_HSVs[0])
TD_HSV = np.array(tone_HSVs[1])
ASD_XYZ = HSV2cone(ASD_HSV)
TD_XYZ = HSV2cone(TD_HSV)

In [15]:
tbs = [np.zeros(len(entropy_grey[0])).reshape(-1,1), np.zeros(len(entropy_grey[1])).reshape(-1,1)]

for j,feature in enumerate(features):
    for i in range(2):
        tbs[i] = np.hstack((tbs[i], np.array(feature[i]).reshape(-1,1)))
tbs = np.vstack((tbs[0][:,1:], tbs[1][:,1:]))

In [18]:
valid_tb = pd.DataFrame(tbs)

valid_tb['X'] = np.hstack((ASD_XYZ[:,0],TD_XYZ[:,0])).reshape(-1,1)
valid_tb['Y'] = np.hstack((ASD_XYZ[:,1],TD_XYZ[:,1])).reshape(-1,1)
valid_tb['Z'] = np.hstack((ASD_XYZ[:,2],TD_XYZ[:,2])).reshape(-1,1)
valid_tb['label'] = np.vstack((np.ones(len(entropy_grey[0]), dtype=np.uint8).reshape(-1,1), np.zeros(len(entropy_grey[1]), dtype=np.uint8).reshape(-1,1)))
valid_tb.columns = ['1D Gray Entropy', '2D Gray Entropy', '1D Hue Entropy', 'Color Richness', 'Relative Area', 'Horizontal Interval Metric', 'Vertical Interval Metric', 'Dominant Color X', 'Dominant Color Y', 'Dominant Color Z', 'label']
valid_tb.to_csv('./results/valid_all_features.csv', index=None)

In [19]:
valid_tb

,1D Gray Entropy,2D Gray Entropy,1D Hue Entropy,Color Richness,Relative Area,Horizontal Interval Metric,Vertical Interval Metric,Dominant Color X,Dominant Color Y,Dominant Color Z,label
0,1.057765,2.355009,2.984306,4.0,0.603691,0.002855,0.004579,-0.000000,0.000000,0.996078,1
1,3.190651,6.889010,4.135668,10.0,0.960619,0.001777,0.002574,0.000000,0.000000,0.988235,1
2,2.765383,6.024832,4.157235,7.0,0.938517,0.002572,0.002980,0.006573,0.004107,0.988235,1
3,2.217748,5.533011,4.632747,5.0,0.695489,0.001402,0.001468,-0.000000,0.000000,0.988235,1
4,2.423091,5.948474,4.300436,7.0,0.745724,0.001381,0.001389,0.002386,0.003054,0.988235,1
5,2.209020,5.404380,4.017312,5.0,0.851555,0.000883,0.001061,0.000000,0.000000,0.992157,1
6,2.673678,6.044106,2.846033,7.0,0.647294,0.002477,0.002281,0.012357,0.014726,0.980392,1
7,2.744171,6.551896,2.198130,4.0,0.901267,0.000720,0.001217,0.002386,0.003054,0.988235,1
8,2.273922,5.194957,3.323777,3.0,0.899065,0.002807,0.003555,0.002603,0.002891,0.992157,1
9,2.263381,5.180726,2.272553,4.0,0.899065,0.002778,0.003407,0.002603,0.002891,0.992157,1
